## Version Description
In this version, we deploy a Hybrid PINN to predict only one correction factor for the MTR ( `MTR-600` ) by the neural network.

In [1]:
from IPython.core.display import display, HTML,display_html
display(HTML("<style>.container { width:95% !important; }</style>"))

In [2]:
# Source: https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [3]:
# import required libraries
import random
import pandas as pd
import numpy as np

from functools import reduce
import sys

#Set some numpy print options for displaying numpy arrays to fit maximum width of cell
np.set_printoptions(precision=3, edgeitems=30, linewidth=1000,formatter=dict(float=lambda x: "%.3g" % x))

# Disable Warnings for chained assignments Eg:Setting with Copy Warning
pd.options.mode.chained_assignment = None

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_file,output_notebook
output_notebook() # Set to output the plot in the notebook

Loading BokehJS ...

In [4]:
# import LTR and corresponding Restrictor dataframe
Restrictor_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_RestrictorData_LTR.csv')
LTR_data = pd.read_csv('D:/OneDrive/Documents/Airbus/GitHub/ML_A350_Restrictors/TZ6/data/Zone6_LTRData.csv')
obs_HOVs = pd.read_csv('../data/LTR_dataset.csv', usecols = ['HoV'])[['HoV']]
LTR_data[['AMBP','MIXP']] *= 100 # Convert hPa to Pa
zero_MIXT_hoV_idx = LTR_data.index[LTR_data['MIXT'] == 20].tolist() # Points where originally, MIXT == 0 and substituted with 20° C
LTR_data[['MIXT','AMBT']] += 273.15 # Convert Celcius to Kelvin
print('Total LTR points:',len(LTR_data))

## Calculate Density = (0.5* Pmix + Pamb) / (R * Tmix) where R = 287 is the Gas constant in Ideal Gas Law
idx = LTR_data.columns.get_loc("AMBT")
LTR_data.insert(loc=(idx+1), column='Density(kg/m³)', value=np.round((0.5 * LTR_data['MIXP'] + LTR_data['AMBP'] ) / (LTR_data['MIXT']*287),4))
target_df = reduce(lambda left,right: pd.merge(left,right,left_on='HoV',right_on='HoV'), [LTR_data,Restrictor_data])
target_df['HoV'] = obs_HOVs.values
target_df.head()

Total LTR points: 34


,HoV,CAOLH_C66-C68,CAOLH_C68-C70,CAOLH_C70-C72,CAOLH_C72-C74,CAOLH_C74-C76,CAOLH_C76-C78,CAORH_C66-C68,CAORH_C68-C70,CAORH_C70-C72,...,R632_HS1,R633_HS1,R634_HS1,R635_HS1,R636_HS1,R637_HS1,R638_HS1,R639_HS1,R640_HS1,R641_HS1
0,A1,36.071000,38.802000,36.796000,36.794000,39.100000,38.318000,37.055000,39.596000,38.333000,...,41,41,41,41,41,41,41,41,41,41
1,A2,36.061000,38.697000,37.156000,36.738000,38.957000,38.326000,36.501000,40.797000,39.284000,...,41,41,41,41,41,41,41,41,41,41
2,A3,37.504234,40.911735,38.412733,37.859887,36.585486,38.504564,36.161971,37.068225,37.893405,...,41,41,41,41,41,41,41,41,41,41
3,A4,36.383413,39.101836,38.110486,37.597728,36.190671,37.706898,36.668551,37.759174,38.497755,...,41,41,41,41,41,41,41,41,41,41
4,A5,36.897000,39.925000,38.704000,38.175000,40.591000,39.913000,37.641000,38.309000,38.600000,...,41,41,41,41,41,41,41,41,41,41


In [5]:
target_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34 entries, 0 to 33
Data columns (total 58 columns):
HoV               34 non-null object
CAOLH_C66-C68     34 non-null float64
CAOLH_C68-C70     34 non-null float64
CAOLH_C70-C72     34 non-null float64
CAOLH_C72-C74     34 non-null float64
CAOLH_C74-C76     34 non-null float64
CAOLH_C76-C78     34 non-null float64
CAORH_C66-C68     34 non-null float64
CAORH_C68-C70     34 non-null float64
CAORH_C70-C72     34 non-null float64
CAORH_C72-C74     34 non-null float64
CAORH_C74-C76     34 non-null float64
CAORH_C76-C78     34 non-null float64
LAOLH_C66-C68     34 non-null float64
LAOLH_C68-C70     34 non-null float64
LAOLH_C70-C72     34 non-null float64
LAOLH_C72-C74     34 non-null float64
LAOLH_C74-C76     34 non-null float64
LAOLH_C76-C78     34 non-null float64
LAORH_C66-C68     34 non-null float64
LAORH_C68-C70     34 non-null float64
LAORH_C70-C72     34 non-null float64
LAORH_C72-C74     34 non-null float64
LAORH_C74-C76     34 non

In [6]:
MIXP = [1800, 2600, 3400]
# Define Restrictor Labels
MTR = 'R600_HD'
CLRS = ['R610_HS1','R611_HS1','R612_HS1','R613_HS1']
CAORS = ['R620_HS1','R620_HS2','R620_HS3','R621_HS1','R621_HS2','R621_HS3']
MHRS = CLRS + CAORS
LAORS = ['R630_HS1','R631_HS1','R632_HS1','R633_HS1','R634_HS1','R635_HS1','R636_HS1','R637_HS1','R638_HS1','R639_HS1','R640_HS1','R641_HS1']

In [7]:
# Duct Areas
MTR_DuctArea = 0.03464 # in sq.m
MHR_Duct_Areas = {'R610_HS1':0.01767,'R611_HS1':0.01767,'R612_HS1':0.02461,'R613_HS1':0.02461,
                 'R620_HS1':0.01090,'R620_HS2':0.00470,'R620_HS3':0.01085,'R621_HS1':0.01085,'R621_HS2':0.00470,'R621_HS3':0.01090}

## Define Initial Correction factor values from CFD
# Correction Factor for Duct Part `TZ6_zmix:
TZ6_zmix_cf = 1 # Start with initial c_f =1 

#3D Correction Factor for MTR
MTR_cf = 1.1664876228437


#3D Correction Factors for MultiHole Restrictors
MHR_cf = {'R610_HS1':0.740773983383846,'R611_HS1':0.740205245723713,'R612_HS1':0.83879652680329,'R613_HS1':0.81775586281569,
          'R620_HS1':0.757121843184036,'R620_HS2':0.773919779037984,'R620_HS3':0.789028593182055,
          'R621_HS1':0.714811824527744,'R621_HS2':0.866271631648556,'R621_HS3':0.822447592395462}

### Calculate Loss-Coefficient $\zeta$
**For Single Hole Restrictor:**<br>

    f0_f1 = A_Circular(Hole_Diameter) / Area_Overall
    l_cross = Thickness / Hole_Diameter
    Zeta_dash = 0.13 + 0.34 * 10 ^ -(3.4 * l_cross + 88.4 * l_cross ^ 2.3)
    Zeta_Single_Hole_Thick_Chamfered = ((1 - f0_f1 + (Zeta_dash ^ 0.5) * (1 - f0_f1) ^ 0.375) ^ 2) * f0_f1 ^ -2

In [8]:
def SHR_Zeta_3D(n_holes,hole_dia,MTR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Single Hole Retrictors
    '''
    MTR_New_Area = n_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MTR_New_Area/MTR_DuctArea
    l_cross = 1/hole_dia
    zeta_dash = 0.13 + 0.34 * 10**(-(3.4 * l_cross + 88.4 * l_cross**2.3))
    zeta_SHR_1D = ((1 - f0_f1 + (zeta_dash**0.5) * (1 - f0_f1)**0.375)**2) * f0_f1**(-2) # 1D Zeta
    zeta_SHR_3D = zeta_SHR_1D * cf # Zeta with 3D Correction Factor    
    return MTR_New_Area,zeta_SHR_3D

In [9]:
target_df[MTR+'_Area'],target_df[MTR+'_Zeta3D'] = zip(*[SHR_Zeta_3D(1,dia,MTR_DuctArea,MTR_cf) for dia in target_df[MTR]])
target_df[[MTR,'R600_HD_Area','R600_HD_Zeta3D']].head()

,R600_HD,R600_HD_Area,R600_HD_Zeta3D
0,148,0.017203,4.949375
1,149,0.017437,4.730821
2,152,0.018146,4.127180
3,154,0.018627,3.764504
4,148,0.017203,4.949375


**For Multi Hole Restrictors:**<br>

    Area_Free = Number_of_Holes * A_Circular(Hole_Diameter)
    f0_f1 = Area_Free / Area_Overall
    l_cross = Thickness / Hole_Diameter
    phi = 0.25 + (0.535 * l_cross ^ 8) / (0.05 + l_cross ^ 7)
    tau = (2.4 - l_cross) * 10 ^ (-phi)
    Zeta_Multi_Hole = (0.5 * (1 - f0_f1) ^ 0.75 + tau * (1 - f0_f1) ^ 1.375 + (1 - f0_f1) ^ 2 + 0.02 * l_cross) / f0_f1 ^ 2

In [10]:
def MHR_Zeta_3D(nr_holes,hole_dia,MHR_DuctArea,cf):
    '''
    Computes the Zeta with 3D Correction Factor (cf) for Multi Hole Restrictors
    '''
    MHR_New_Area = nr_holes * (np.pi/4) * (hole_dia / 1000)**2 # Divide dia by 1000 to convert mm to m
    f0_f1 = MHR_New_Area/MHR_DuctArea
    l_cross = (0.00144*1000)/hole_dia
    phi = 0.25 + (0.535 * l_cross**8) / (0.05 + l_cross**7)
    tau = (2.4 - l_cross) * 10**(-phi)
    zeta_MHR_1D = (0.5 * (1 - f0_f1)**0.75 + tau * (1 - f0_f1)**1.375 + (1 - f0_f1)**2 + 0.02 * l_cross) / f0_f1**2 # 1D Zeta     
    zeta_MHR_3D = zeta_MHR_1D * cf # Zeta with 3D Correction Factor    
    return MHR_New_Area,zeta_MHR_3D

In [11]:
# Calculate Zeta for Multi-Hole Restrictors
for clr in CLRS:
    if (clr == 'R610_HS1') | (clr == 'R611_HS1'):
        hole_dia = 8
    else:
        hole_dia = 10
    MHR_nr_holes = target_df[clr].values
    target_df[clr+'_Area'], target_df[clr+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,hole_dia,MHR_Duct_Areas[clr],MHR_cf[clr]) for ele in MHR_nr_holes])
    
for caor in CAORS:
    CAOR_nr_holes = target_df[caor].values
    target_df[caor+'_Area'],target_df[caor+'_Zeta3D'] = zip(*[MHR_Zeta_3D(ele,8,MHR_Duct_Areas[caor],MHR_cf[caor]) for ele in CAOR_nr_holes])

In [12]:
# LTR Dataframe col index for Restrictor Zeta and Area values
target_df.iloc[:,58:]

,R600_HD_Area,R600_HD_Zeta3D,R610_HS1_Area,R610_HS1_Zeta3D,R611_HS1_Area,R611_HS1_Zeta3D,R612_HS1_Area,R612_HS1_Zeta3D,R613_HS1_Area,R613_HS1_Zeta3D,...,R620_HS2_Area,R620_HS2_Zeta3D,R620_HS3_Area,R620_HS3_Zeta3D,R621_HS1_Area,R621_HS1_Zeta3D,R621_HS2_Area,R621_HS2_Zeta3D,R621_HS3_Area,R621_HS3_Zeta3D
0,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
1,0.017437,4.730821,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
2,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.008954,9.156844,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
3,0.018627,3.764504,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
4,0.017203,4.949375,0.007288,5.595443,0.007691,4.756868,0.010210,6.271347,0.010210,6.114035,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
5,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
6,0.017203,4.949375,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
7,0.018385,3.942093,0.006585,7.506119,0.007037,6.198696,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
8,0.018146,4.127180,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009817,6.862828,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119
9,0.017671,4.521189,0.006585,7.506119,0.006836,6.741078,0.009425,7.920333,0.009425,7.721657,...,0.00191,6.107864,0.00568,2.786638,0.003770,8.801675,0.00191,6.836716,0.00568,2.95119


### Run FDDN Solver

In [13]:
from FDDN_Lib import fddn_zeta_input, FDDN_Solver, FDDN_output_df_gen, name_variables, zeta_values

In [14]:
print(name_variables)

['TZ-6_zmix', 'MTR-600', 'riser_LAO_RHS', 'riser_LAO_LHS', 'riser_CAO_RHS', 'riser_CAO_LHS', 'R-610', 'R-611', 'R-612', 'R-613', 'LAOR-630', 'LAOR-632', 'LAOR-634', 'LAOR-636', 'LAOR-638', 'LAOR-640', 'LAOR-631', 'LAOR-633', 'LAOR-635', 'LAOR-637', 'LAOR-639', 'LAOR-641', 'CAO-620_fwd', 'CAO-620_mid', 'CAO-620_aft', 'CAO-621_fwd', 'CAO-621_mid', 'CAO-621_aft', 'LAO-630', 'LAO-632', 'LAO-634', 'LAO-636', 'LAO-638', 'LAO-640', 'LAO-631', 'LAO-633', 'LAO-635', 'LAO-637', 'LAO-639', 'LAO-641', 'CAOR-620_fwd', 'CAOR-620_mid', 'CAOR-620_aft', 'CAOR-621_fwd', 'CAOR-621_mid', 'CAOR-621_aft', 'pp2610pdloss', 'pp2611pdloss', 'pp2612pdloss', 'pp2613pdloss', '26120-R620_F', '26120-R620_M', '26120-R620_A', '26121-R621_F', '26121-R621_M', '26121-R621_A']


In [15]:
# Gather col_names and their corresponding column indices
zeta_col_names = [col for col in target_df.columns if 'Zeta3D' in col]
zeta_col_names_idx = [target_df.columns.get_loc(col) for col in zeta_col_names if col in target_df]
area_col_names = [col for col in target_df.columns if 'Area' in col]
area_col_names_idx = [target_df.columns.get_loc(col) for col in area_col_names if col in target_df]
pt_var_idx = [target_df.columns.get_loc(col) for col in ['MIXP','AMBP','AMBT'] if col in target_df]
req_flow_rates_idx = [target_df.columns.get_loc(col) for col in ['TZ6_Flow'] if col in target_df]
# print(zeta_col_names_idx)
print(zeta_col_names)

['R600_HD_Zeta3D', 'R610_HS1_Zeta3D', 'R611_HS1_Zeta3D', 'R612_HS1_Zeta3D', 'R613_HS1_Zeta3D', 'R620_HS1_Zeta3D', 'R620_HS2_Zeta3D', 'R620_HS3_Zeta3D', 'R621_HS1_Zeta3D', 'R621_HS2_Zeta3D', 'R621_HS3_Zeta3D']


In [16]:
LTR_zeta_df_col_idx = sorted([0] + req_flow_rates_idx + pt_var_idx + zeta_col_names_idx)
target_df.iloc[:,LTR_zeta_df_col_idx].to_csv('data_output/LTR_Zeta_df.csv', index=False)

In [17]:
LTR_zeta_df = target_df[['HoV','MIXP','AMBP','AMBT','TZ6_Flow']+zeta_col_names]
# LTR_zeta_df.tail()

In [18]:
final_df = target_df[['HoV','MIXP','Density(kg/m³)','R600_HD','R600_HD_Zeta3D','TZ6_Flow']]
final_df.reset_index(drop=True,inplace = True)

### Training data Preparation

In [19]:
from Hybrid_PINN_v1 import NormbyMax, DeNormbyMax, PINN

In [20]:
input_features = ['MIXP','Density(kg/m³)','R600_HD_Zeta3D','TZ6_Flow']
features_max, df_rescaled = NormbyMax(final_df,input_features)
print('Max value of all features:',features_max)

V_max_org = features_max['TZ6_Flow']
print('Maximum TZ6 Flow:',V_max_org)

Max value of all features: {'MIXP': 2632.9689, 'Density(kg/m³)': 1.238, 'R600_HD_Zeta3D': 5.4148802645163, 'TZ6_Flow': 764.0123719999998}
Maximum TZ6 Flow: 764.0123719999998


In [21]:
# Display original and Scaled DataFrame Side-by-side
# print( '{:<60s} {:<20s}'.format('Original Input Dataframe:','Rescaled Input Dataframe:') )
# display_side_by_side(final_df[['HoV']+input_features],df_rescaled)

In [22]:
df_rescaled_inv = DeNormbyMax(df_rescaled,features_max,input_features)

In [23]:
train_data = df_rescaled
# train_data = df_rescaled.iloc[[7,15]]
print("Number of training samples:", len(train_data))
train_data

Number of training samples: 34


,MIXP,Density(kg/m³),R600_HD_Zeta3D,TZ6_Flow
0,0.987478,0.986026,0.914032,0.966062
1,0.987478,0.987722,0.873670,0.965658
2,0.989830,0.976575,0.762192,0.969853
3,0.987440,0.999031,0.695215,0.958246
4,0.987478,1.000000,0.914032,0.970963
5,0.987478,0.985299,0.914032,0.973195
6,0.987478,0.978837,0.914032,0.968664
7,0.988183,0.967367,0.728011,0.971982
8,0.985737,0.960420,0.762192,0.968232
9,0.987478,0.996446,0.834956,0.968331


In [24]:
# Store indices and values of train data
train_data_idx = train_data.index.values.tolist()
train_R600HD_series = final_df['R600_HD'].iloc[train_data_idx]
X_train = train_data.values

### Predict $c_f$ with Physics Informed Neural Network
A random seed is used to reproduce the weight initialization values and training performance everytime this notebook is run.

In [27]:
np.random.seed(42)
# Define custom MSE function
MSE = lambda y_hat,y_true: np.mean((y_hat-y_true)**2)

**Train the Neural Network**:

In [28]:
### Set the hyperparameters here ###
iterations = 100
learning_rate = 0.01 ### Important Parameter
hidden_nodes = 2 ### Important Parameter
output_nodes = 1
MTR_epsi = MTR_cf * 0.01

In [29]:
N_i = X_train.shape[1]
network = PINN(N_i, hidden_nodes, output_nodes, learning_rate)

train_loss_hist = []
losses_hist = {'HoV':[],'train_loss':[]}
c_f_hist = {'HoV':[],'c_f_hat':[]}

for idx in train_data_idx:    
    ## Stochastic Gradient Descent with mini-batch training
    print('Old_Zeta:',target_df[MTR+'_Zeta3D'].loc[idx])
    for ii in range(iterations):    
        # Go through one record at a time from the training data set
        X_train = train_data[input_features].loc[[idx]].values
        dia_train = np.atleast_1d(train_R600HD_series.loc[idx])    
        HoV,V_hat_FDDN,V_true_LTR,break_flag,c_f = network.train(X_train,dia_train,[idx],MTR_epsi,MTR_DuctArea,target_df,zeta_col_names,V_max_org)

        # Printing out the training progress
        train_loss = MSE(V_hat_FDDN,V_true_LTR) # MSE between FDDN predicted flow rate and true flow rate on UN-scaled original values
        sys.stdout.write("\rProgress: {:2.1f}".format(100 * ii/float(iterations))+ "% ... Training loss: " + str(round(train_loss,4))[:5])
        print('... Training Iteration:',ii)    
        sys.stdout.flush()
        train_loss_hist.append(train_loss)       
        if (break_flag == 1) or (ii == (iterations - 1)): # early stopping
            print('EARLY STOPPING ACTIVATED - Terminating Neural Network Training')
            print('Train Loss History for {}:'.format(HoV[0]),train_loss_hist)
            losses_hist['HoV'].append(HoV[0])
            losses_hist['train_loss'].append(train_loss_hist)
            c_f_hist['HoV'].append(HoV[0])
            c_f_hist['c_f_hat'].append(c_f[0])
            train_loss_hist = []
            break

Old_Zeta: 4.949375283472444

NN output is -ve. Taking |(c_f)| value: [[0.0421]]
New_Zeta: [0.179]
FDDN Solver Output: [[99.8 47.6 150 98.4 52 160 15.7 33.9 33.9 33.9 33.2 32.8 15.3 33.4 33.7 33.1 32.7 32.7]]
New_Zeta_epsi: [-0.228]
FDDN Solver Output: [[99.3 47.4 150 98 51.7 160 15.7 33.8 33.7 33.8 33.1 32.6 15.3 33.3 33.5 33 32.6 32.6]]
Row ID: [0]
HoV: ['A1']
FlowRate Difference (LTR - FDDN): [-235] l/s
Progress: 0.0% ... Training loss: 55202... Training Iteration: 0

NN output (C_f): [3.62]
New_Zeta: [15.3]
FDDN Solver Output: [[51.9 24.8 78.3 51.2 27.1 83.5 8.21 17.7 17.7 17.7 17.3 17.1 8.01 17.4 17.6 17.3 17.1 17.1]]
New_Zeta_epsi: [15.4]
FDDN Solver Output: [[51.9 24.7 78.2 51.1 27 83.4 8.19 17.7 17.7 17.7 17.3 17.1 8 17.4 17.6 17.3 17.1 17.1]]
Row ID: [0]
HoV: ['A1']
FlowRate Difference (LTR - FDDN): [231] l/s
Progress: 1.0% ... Training loss: 53425... Training Iteration: 1

NN output (C_f): [2.03]
New_Zeta: [8.63]
FDDN Solver Output: [[63.1 30.1 95.1 62.2 32.9 101 9.96 21.5 21.

FDDN Solver Output: [[76.1 36.3 115 75.1 39.7 122 12 25.9 25.9 25.9 25.4 25 11.7 25.5 25.7 25.3 25 25]]
New_Zeta_epsi: [4.07]
FDDN Solver Output: [[75.9 36.2 114 74.9 39.6 122 12 25.9 25.8 25.9 25.3 25 11.7 25.5 25.7 25.3 24.9 25]]
Row ID: [5]
HoV: ['A6']
FlowRate Difference (LTR - FDDN): [0.724] l/s
% Diff between flowrates: [0.0974]
Progress: 2.0% ... Training loss: 0.524... Training Iteration: 2
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for A6: [82.62779513999176, 10.633219042013732, 0.5243485323553908]
Old_Zeta: 4.949375283472444

NN output (C_f): [0.935]
New_Zeta: [3.97]
FDDN Solver Output: [[76.8 36.6 116 75.7 40 123 12.1 26.1 26.1 26.2 25.6 25.2 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [4.02]
FDDN Solver Output: [[76.6 36.5 115 75.5 39.9 123 12.1 26.1 26 26.1 25.5 25.2 11.8 25.7 25.9 25.5 25.1 25.2]]
Row ID: [6]
HoV: ['C1']
FlowRate Difference (LTR - FDDN): [-8.8] l/s
Progress: 0.0% ... Training loss: 77.51... Training Iteration: 0




NN output (C_f): [1.04]
New_Zeta: [3.86]
FDDN Solver Output: [[77.4 37 117 76.4 40.3 124 12.2 26.4 26.3 26.4 25.8 25.5 11.9 26 26.2 25.8 25.4 25.5]]
New_Zeta_epsi: [3.91]
FDDN Solver Output: [[77.3 36.9 116 76.2 40.3 124 12.2 26.3 26.3 26.3 25.8 25.4 11.9 25.9 26.1 25.7 25.4 25.4]]
Row ID: [12]
HoV: ['D1']
FlowRate Difference (LTR - FDDN): [-1.57] l/s
% Diff between flowrates: [0.208]
Progress: 1.0% ... Training loss: 2.465... Training Iteration: 1
EARLY STOPPING ACTIVATED - Terminating Neural Network Training
Train Loss History for D1: [681.8152842064761, 2.4651465226505995]
Old_Zeta: 4.949375283472444

NN output (C_f): [1.04]
New_Zeta: [4.43]
FDDN Solver Output: [[77.1 36.3 111 76.8 39.5 118 11.7 25.2 25.1 25.2 24.7 24.3 11.4 24.8 25 24.6 24.3 24.3]]
New_Zeta_epsi: [4.48]
FDDN Solver Output: [[76.9 36.2 111 76.6 39.4 118 11.6 25.1 25.1 25.1 24.6 24.3 11.4 24.7 24.9 24.5 24.2 24.2]]
Row ID: [13]
HoV: ['D2']
FlowRate Difference (LTR - FDDN): [7.93] l/s
Progress: 0.0% ... Training loss

FDDN Solver Output: [[68.2 32.6 103 67.3 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: [15]
HoV: ['E2']
FlowRate Difference (LTR - FDDN): [3.36] l/s
Progress: 13.0% ... Training loss: 11.29... Training Iteration: 13

NN output (C_f): [1.51]
New_Zeta: [6.41]
FDDN Solver Output: [[68.4 32.6 103 67.4 35.6 110 10.8 23.3 23.3 23.3 22.8 22.5 10.5 22.9 23.1 22.8 22.5 22.5]]
New_Zeta_epsi: [6.46]
FDDN Solver Output: [[68.3 32.6 103 67.3 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: [15]
HoV: ['E2']
FlowRate Difference (LTR - FDDN): [3.27] l/s
Progress: 14.0% ... Training loss: 10.69... Training Iteration: 14

NN output (C_f): [1.51]
New_Zeta: [6.41]
FDDN Solver Output: [[68.4 32.6 103 67.4 35.6 110 10.8 23.3 23.3 23.3 22.8 22.5 10.5 22.9 23.1 22.8 22.5 22.5]]
New_Zeta_epsi: [6.46]
FDDN Solver Output: [[68.2 32.6 103 67.3 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: [15]
HoV: ['E2']
FlowRate Diff


NN output (C_f): [1.51]
New_Zeta: [6.41]
FDDN Solver Output: [[68.4 32.6 103 67.4 35.6 110 10.8 23.3 23.3 23.3 22.8 22.5 10.5 22.9 23.1 22.8 22.5 22.5]]
New_Zeta_epsi: [6.46]
FDDN Solver Output: [[68.2 32.6 103 67.3 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: [15]
HoV: ['E2']
FlowRate Difference (LTR - FDDN): [3.34] l/s
Progress: 33.0% ... Training loss: 11.12... Training Iteration: 33

NN output (C_f): [1.51]
New_Zeta: [6.41]
FDDN Solver Output: [[68.4 32.6 103 67.4 35.6 110 10.8 23.3 23.3 23.3 22.8 22.5 10.5 22.9 23.1 22.8 22.5 22.5]]
New_Zeta_epsi: [6.46]
FDDN Solver Output: [[68.2 32.6 103 67.3 35.6 110 10.8 23.3 23.2 23.3 22.8 22.5 10.5 22.9 23.1 22.7 22.4 22.4]]
Row ID: [15]
HoV: ['E2']
FlowRate Difference (LTR - FDDN): [3.33] l/s
Progress: 34.0% ... Training loss: 11.07... Training Iteration: 34

NN output (C_f): [1.51]
New_Zeta: [6.41]
FDDN Solver Output: [[68.4 32.6 103 67.4 35.6 110 10.8 23.3 23.3 23.3 22.8 22.5 10.5 22.9 23.1 22.8 22.5 22.

...

FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.27] l/s
Progress: 22.0% ... Training loss: 18.21... Training Iteration: 22

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.24] l/s
Progress: 23.0% ... Training loss: 18.00... Training Iteration: 23

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Differen


NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.28] l/s
Progress: 42.0% ... Training loss: 18.28... Training Iteration: 42

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.2] l/s
Progress: 43.0% ... Training loss: 17.66... Training Iteration: 43

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New

FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.24] l/s
Progress: 61.0% ... Training loss: 18.00... Training Iteration: 61

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.24] l/s
Progress: 62.0% ... Training loss: 18.00... Training Iteration: 62

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Differen


NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.27] l/s
Progress: 81.0% ... Training loss: 18.21... Training Iteration: 81

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
New_Zeta_epsi: [3.82]
FDDN Solver Output: [[77.8 37.1 117 75.4 39.8 123 12.3 26.5 26.4 26.5 25.9 25.6 11.8 25.6 25.9 25.4 25.1 25.1]]
Row ID: [31]
HoV: ['S5']
FlowRate Difference (LTR - FDDN): [4.24] l/s
Progress: 82.0% ... Training loss: 18.00... Training Iteration: 82

NN output (C_f): [1.56]
New_Zeta: [3.79]
FDDN Solver Output: [[77.9 37.2 117 75.5 39.9 123 12.3 26.5 26.5 26.5 26 25.6 11.8 25.7 25.9 25.5 25.2 25.2]]
Ne

FDDN Solver Output: [[71.5 34.1 108 70.5 37.2 115 11.3 24.3 24.3 24.4 23.8 23.5 11 24 24.2 23.8 23.5 23.5]]
New_Zeta_epsi: [5.63]
FDDN Solver Output: [[71.3 34 107 70.3 37.1 115 11.3 24.3 24.2 24.3 23.8 23.5 11 23.9 24.1 23.7 23.4 23.4]]
Row ID: [32]
HoV: ['T1']
FlowRate Difference (LTR - FDDN): [25.2] l/s
Progress: 0.0% ... Training loss: 635.4... Training Iteration: 0

NN output (C_f): [1.09]
New_Zeta: [4.63]
FDDN Solver Output: [[74.7 35.7 113 73.7 38.9 120 11.8 25.5 25.4 25.5 24.9 24.6 11.5 25.1 25.3 24.9 24.6 24.6]]
New_Zeta_epsi: [4.68]
FDDN Solver Output: [[74.6 35.6 112 73.5 38.8 120 11.8 25.4 25.4 25.4 24.9 24.5 11.5 25 25.2 24.8 24.5 24.5]]
Row ID: [32]
HoV: ['T1']
FlowRate Difference (LTR - FDDN): [-6.76] l/s
Progress: 1.0% ... Training loss: 45.70... Training Iteration: 1

NN output (C_f): [1.15]
New_Zeta: [4.9]
FDDN Solver Output: [[73.8 35.2 111 72.8 38.4 119 11.6 25.1 25.1 25.1 24.6 24.3 11.4 24.8 25 24.5 24.2 24.2]]
New_Zeta_epsi: [4.95]
FDDN Solver Output: [[73.6 35.1 

In [30]:
cf_df = pd.DataFrame(data=c_f_hist)
losses = pd.DataFrame(data=losses_hist)

In [31]:
cf_df['MaxEpochs'] = None # Initialize Empty Column for storing epochs trained

for i in range(len(losses['HoV'])):
    nr_epochs_trained = len(losses['train_loss'][i])
    cf_df['MaxEpochs'].loc[i] = nr_epochs_trained

cf_df

,HoV,c_f_hat,MaxEpochs
0,A1,1.010093,5
1,A2,1.071597,2
2,A3,1.159768,2
3,A4,1.301815,2
4,A5,0.988150,4
5,A6,0.948585,3
6,C1,0.981955,3
7,C2,1.275775,3
8,C3,1.250590,2
9,C4,1.033093,2


In [32]:
print('Max no. of epochs for:',cf_df['HoV'].iloc[[cf_df.MaxEpochs.idxmax()]].values)
cf_df.to_csv('data_output/NN_MTR_CF_Output.csv',index=False)
cf_df.iloc[[cf_df.MaxEpochs.idxmax()]]

Max no. of epochs for: ['E2']


,HoV,c_f_hat,MaxEpochs
15,E2,1.51072,100


**Plot Training Loss**

In [33]:
from bokeh.palettes import Category20,Colorblind,Spectral,Set2,YlGnBu,RdPu
temp_list = []
bokeh_palettes = [Colorblind,YlGnBu,RdPu,Set2]
for palette in bokeh_palettes:
    for key in palette.keys():
        temp_list.append(palette[key])
    
color_palette = [y for x in temp_list for y in x]

In [34]:
num_lines = len(losses.HoV) # no. of lines to draw
colors = color_palette[0:num_lines]
labels = losses.HoV.values.tolist()

p = figure(width=1100)
for i in range(num_lines):
    x = list(range(1,len(losses['train_loss'][i])+1))
    y = losses['train_loss'][i]
    p.line(x, y, line_width=2, color=colors[i], alpha=0.8, legend='Train Loss for {}'.format(labels[i]))
p.yaxis.axis_label = "Train Loss"
p.xaxis.axis_label = "Epochs"
p.legend.click_policy="hide"
show(p)

### Observations:
In general the performance of the Hybrid PINN architecture seems to be satisfactory and the final `c_f` predicted is physically consistent. Some drawbacks that were observed are:
* The output of the neural network sometimes stagnates for many training iterations without any significant improvement (Eg: as in the case of HoVs: **E2** and **S5** )
* The early stopping criteria is specified in the `Hybrid_PINN_v1.py` script which uses a percentage difference (set to 0.35%) between the LTR and FDDN Flowrate. Changing this value also affects the training performance of the neural network.